# CSC 8614 - Language Models
## CI2 - Fine-tuning a language model for text classification

In this TP, you will work on fine-tuning a language model to move from text generation to text classification, specifically working on Spam Detection.

The exercise (and code) has been adapted from the book _Build a Large Language Model (From Scratch)_, by Sebastian Raschka, and its [official github repository](https://github.com/rasbt/LLMs-from-scratch).

This TP will be done in this notebook, and requires some additional files (available from the course website). You will have to fill the missing portions of code, and perform some additional experiments by testing different parameters.

Working on this TP:
- The easiest way is probably to work directly on the notebook, using jupyter notebook or visual studio code. An alternative is also to use Google colab.
- You should be able to run everything on your machine, but you can connect to the GPUs if needed.

Some files are required, and are available on the course website:
- `requirements.txt`
- `gpt_utils.py`


## About the report
You will have to return this notebook (completed), as well as a mini-report (`TP2/rapport.md`).

The notebook and report shall be submitted via a GitHub repository, similarly to what you did for the first session (remember to use a different folder: `TP2`).
For the notebook, it is sufficient to complete the code and submit the final version.

For the mini-report, you have to answer the questions asked in this notebook, and discuss some of your findings as requested.
As for the first session:
- "Vous devez y mettre : réponses courtes, résultats observés (copie de sorties), captures d’écran demandées, et une courte interprétation."
- "Ne collez pas des pages entières : soyez concis et sélectionnez les éléments pertinents."

Reproducibility: 
- fix a random seed and write it in the report
- indicate in the report the specific python version OS, and the library versions.

**Question 1**: Dans `TP1/rapport.md`, ajoutez immédiatement un court en-tête (quelques lignes) contenant : (i) votre nom/prénom, (ii) la commande d’installation/activation d’environnement utilisée, (iii) les versions (Python + bibliothèques principales).

Ajoutez ensuite au fil du TP des sections/titres à votre convenance, tant que l’on peut retrouver clairement vos réponses et vos preuves d’exécution.

In [1]:
# [Instructor code: install requirements]
!pip install -r requirements.txt

  Using cached tensorflow-2.20.0-cp312-cp312-win_amd64.whl.metadata (4.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.12.19-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached gast-0.7.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached wrapt-2.0.1-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
  Using cached keras-3.13.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached h5py-3.15.1-cp312-cp312-win_amd64.whl.metadata (3.1 kB)
  Using cached ml_dtypes-0.5.4-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached rich-14.2.0-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.1.0-py3-


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Preparing the model

In [2]:
# --- [INSTRUCTOR CODE: load the model weights into memory] ---
import torch
import tiktoken
from gpt_utils import GPTModel, download_and_load_gpt2, load_weights_into_gpt

# Download the model weights (124M param version) / This function (which we put in gpt_utils) handles the downloading
settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2_weights")
print("Weights downloaded and loaded into memory.")

c:\Users\ahmed\Desktop\CSC8614\venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
checkpoint: 100%|██████████| 77.0/77.0 [00:00<00:00, 14.4kiB/s]
encoder.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 1.52MiB/s]
hparams.json: 100%|██████████| 90.0/90.0 [00:00<00:00, 89.6kiB/s]
model.ckpt.data-00000-of-00001: 100%|██████████| 498M/498M [01:14<00:00, 6.71MiB/s] 
model.ckpt.index: 100%|██████████| 5.21k/5.21k [00:00<00:00, 4.38MiB/s]
model.ckpt.meta: 100%|██████████| 471k/471k [00:00<00:00, 843kiB/s] 
vocab.bpe: 100%|██████████| 456k/456k [00:00<00:00, 760kiB/s] 


Weights downloaded and loaded into memory.


The `settings` obtained with `download_and_load_gpt2` are the GPT-2 weights made publicly available by OpenAI.

**Question 2**: What type is the object `setting`, and what is its structure (e.g. if it is a list, its length; if a dictionary, its keys, etc.)?

**Question 3**: What type is the object `params`, and what is its structure?

In [6]:
# Analyse `settings`
print("Settings type:", type(settings))
print("Settings keys:", settings.keys())
print("Settings values:", settings.values())


# Analyse `params`
print("Params type:", type(params))
print("Params keys:", params.keys())
print("Shape of 'wpe':", params['wpe'].shape)
print("Shape of first block's attention weights 'w':", params['blocks'][0]['attn']['c_attn']['w'].shape)

Settings type: <class 'dict'>
Settings keys: dict_keys(['n_vocab', 'n_ctx', 'n_embd', 'n_head', 'n_layer'])
Settings values: dict_values([50257, 1024, 768, 12, 12])
Params type: <class 'dict'>
Params keys: dict_keys(['blocks', 'b', 'g', 'wpe', 'wte'])
Shape of 'wpe': (1024, 768)
Shape of first block's attention weights 'w': (768, 2304)


Look at the `GPTModel` in the file `gpt_utils.py`. In the `__init__` method, we have to pass a config (parameter `cfg`). 

**Question 4:** 
Analyse the `__init__` method, and check what is the required structure for the `cfg` parameter. Is the `settings` variable we have obtained in the right format? If not, perform the mapping to convert the variable `setting` into a variable `model_config` with the right structure.

In [7]:
model_config = {
    "vocab_size": settings["n_vocab"],
    "emb_dim": settings["n_embd"],
    "context_length": settings["n_ctx"],
    "n_layers": settings["n_layer"],
    "n_heads": settings["n_head"],
    "drop_rate": 0.1,
    "qkv_bias": True,
}

In [8]:
model = GPTModel(model_config)

# Load the pre-trained weights
load_weights_into_gpt(model, params)
model.eval() 

print("GPT-2 Model Loaded and Configured successfully!")

GPT-2 Model Loaded and Configured successfully!


## Preparing the data

Context from the lecture: The raw data is just text messages. 

The model needs numbers (token IDs). We also need to pad the messages so they are all the same length in a batch.

We will use a `SpamDataset` class (provided below) to tokenize the text.

In [29]:
# --- [INSTRUCTOR CODE: Run this cell to define the Dataset Class] ---
from torch.utils.data import Dataset
import pandas as pd
import urllib.request
import zipfile
import os

class SpamDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=120, pad_token_id=50256):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.pad_token_id = pad_token_id
        # Encode labels: "spam" -> 1, "ham" -> 0
        self.data["label_encoded"] = self.data["Label"].map({"spam": 1, "ham": 0})

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]["Text"]
        label = self.data.iloc[idx]["label_encoded"]
        # Tokenize
        encoded = self.tokenizer.encode(text, allowed_special={'<|endoftext|>'})       
        # Truncate if too long
        encoded = encoded[:self.max_length]
        # Pad if too short
        pad_len = self.max_length - len(encoded)
        encoded += [self.pad_token_id] * pad_len
        attn_mask = [1] * (self.max_length - pad_len) + [0] * pad_len
        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(attn_mask, dtype=torch.long),
            torch.tensor(label, dtype=torch.long),
        )


# Download the dataset zip file
url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
zip_path = "sms_spam_collection.zip"
extract_path = "sms_spam_collection"
data_file_path = os.path.join(extract_path, "SMSSpamCollection")
if not os.path.exists(zip_path):
    print("Downloading dataset...")
    urllib.request.urlretrieve(url, zip_path)
    print("Download complete.")
# Unzip
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_path)
# Read the TSV file
df = pd.read_csv(
    data_file_path, 
    sep="\t", 
    header=None, 
    names=["Label", "Text"]
)
print(f"Total samples loaded: {len(df)}")

# 4. Create Train/Test Split (80 train / 20 test)
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
# Split index
split_idx = int(0.8 * len(df))

# TODO: if needed (for performance resons), you can come back here and reduce the size of the training set.
train_df = df.iloc[:split_idx]  # [:2000]  # Readd this to only consider 2000 training samples
test_df = df.iloc[split_idx:]

# Save as CSVs, so the SpamDataset class can read them.
train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)
print("Created 'train.csv' and 'test.csv' successfully!")
print(f"Train size: {len(train_df)}")
print(f"Test size: {len(test_df)}")

Total samples loaded: 5572
Created 'train.csv' and 'test.csv' successfully!
Train size: 4457
Test size: 1115


**Question 5.1**: In the cell above, why did we do `df = df.sample(frac=1, random_state=123)` when creating the train/test split?

**Question 5.2**: Analyse the datasets, what is the distribution of the two classes in the train set? Are they balanced or unbalanced? In case they are unbalanced, might this lead to issues for the fine-tuning of the model?

In [30]:
print(train_df['Label'].value_counts())
class_distribution = train_df['Label'].value_counts()


Label
ham     3860
spam     597
Name: count, dtype: int64


**Question 6**: Create the dataloaders for training and test.

In [31]:
# TODO: add any imports which are needed
from torch.utils.data import Dataset, DataLoader

# Create the Tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

# Instantiate the Dataset
train_dataset = SpamDataset("train.csv", tokenizer)
test_dataset = SpamDataset("test.csv", tokenizer)

# --- TODO: Create DataLoaders ---
# 1. Create a train_loader with batch_size=16 and shuffle=True
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# 2. Create a test_loader with batch_size=16 and shuffle=False
test_loader =  DataLoader(test_dataset, batch_size=16, shuffle=False)

In [35]:
# Check your work
for input_batch, attn_mask_batch, target_batch in train_loader:
    print("Input batch shape:", input_batch.shape)        # [16, 120]
    print("Attention mask shape:", attn_mask_batch.shape)  # [16, 120]
    print("Target batch shape:", target_batch.shape)      # [16]
    break


Input batch shape: torch.Size([16, 120])
Attention mask shape: torch.Size([16, 120])
Target batch shape: torch.Size([16])


**Question 7**: Looking at the batch size and the training size, how many batches will you have in total? Please report the size of the subsampled training data, you reduce it due to performance constraints.

In [36]:

print("\nAnswer to Q7: Calculating the number of batches...")
num_batches = len(train_loader)
print(f"The training dataset has {len(train_df)} samples.")
print(f"With a batch size of 16, there are {num_batches} batches in the train_loader.")
print(f"Calculation: ceil({len(train_df)} / 16) = {num_batches}")



Answer to Q7: Calculating the number of batches...
The training dataset has 4457 samples.
With a batch size of 16, there are 279 batches in the train_loader.
Calculation: ceil(4457 / 16) = 279


## Fine-tuning

**Context**: GPT-2 was trained to predict the next word (output size ~50,000). We want to predict binary classes (output size 2), so we must replace the final layer.

**Question 8**:

**8.1**: In the cell below, define the number of output classes (`num_classes`) for the new spam detection task.

**8.2**: Also, pring the original and updated output heads (hint: `out_head` from `GPTModel`)

**8.3**: Why do we freeze the internal layers with `param.requires_grad = False`?

In [37]:
# Freeze the internal layers
for param in model.parameters():
    param.requires_grad = False

print(f"Original output head: {model.out_head}")

print("\nAnswer to Q8.3: We freeze the internal layers to leverage transfer learning. The pre-trained GPT-2 model has already learned a powerful representation of the English language. By freezing its layers, we keep this knowledge intact and only train the new classification head and fine-tune the very last layer norm. This is much more efficient than training the whole model from scratch, requires significantly less data, and helps prevent overfitting on our small dataset.")

num_classes = 2
model.out_head = torch.nn.Linear(in_features=model_config["emb_dim"], out_features=num_classes)
# Hint: The input size of the last layer in GPT-2 small is 768.

# Enable gradient calculation ONLY for the new head and the final LayerNorm
for param in model.out_head.parameters():
    param.requires_grad = True
for param in model.trf_blocks[-1].norm2.parameters():
    param.requires_grad = True

print(f"New output head: {model.out_head}")

Original output head: Linear(in_features=768, out_features=2, bias=True)

Answer to Q8.3: We freeze the internal layers to leverage transfer learning. The pre-trained GPT-2 model has already learned a powerful representation of the English language. By freezing its layers, we keep this knowledge intact and only train the new classification head and fine-tune the very last layer norm. This is much more efficient than training the whole model from scratch, requires significantly less data, and helps prevent overfitting on our small dataset.
New output head: Linear(in_features=768, out_features=2, bias=True)


You now have to **finalise the code for the training loop** (see individual steps below).

In the first cell below you can find the code to move the model to GPU (if available), define the optimizer, and calculate the accuracy. The following cell contains the code for the training (fine-tuning) loop.

You will have to complete the code of the training loop, by answering the following questions:

**Question 9.1**: Reset the gradients of the `optimizer` ([hint](https://docs.pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html)).

**Question 9.2**: Compute cross-entropy loss ([hint](https://docs.pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html)).

**Question 9.3**: Add code for the backward pass, to compute the gradient ([hint](https://docs.pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html))

**Question 9.4**: Add code for the optimizer step, to update the weights ([hint](https://docs.pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html))

**Question 9.5**: Add code to calculate the accuracy on train and test (hint: you can use the `calc_accuracy` method).

**Note about the speed**: On my laptop's CPU 1 epoch with the full training dataset (~4400 samples, batch_size=16) took ~20 minutes; 1 epoch with a train set of 2000 samples (batch_size=16) took ~12 minutes. 

To iterate more quickly, you could:
- i) set `num_epochs = 1` (but only at the beginning), just to make sure that the code is working;
- ii) increase batch_size to 32 or 64 (but careful with possible memory issues).
- iii) reduce the size of the training dataset, by going back to the *Preparing the data* section, and changing the line `train_df = df.iloc[:split_idx]` to `train_df = df.iloc[:split_idx][:2000]` or similar. Be careful that if you reduce the training data too much, the model will not have enough data for fine-tuning.
- Use a GPU; it would be much quicker (few minutes on the whole training data).


In [38]:
# [--- INSTRUCTOR CODE ---]
import torch.nn.functional as F

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Measure imbalance
count_ham = len(train_df[train_df['Label']=='ham'])
count_spam = len(train_df[train_df['Label']=='spam'])

# Calculate weight: penalize missing the minority class (Spam) more
# Weight = Count(Majority) / Count(Minority)
pos_weight = (count_ham / count_spam) ** 0.5   # sqrt ratio (less aggressive)
class_weights = torch.tensor([1.0, pos_weight]).to(device)

print(f"Using class weights: {class_weights}")

# Define Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.1)

# Calculate Accuracy Helper Function
def calc_accuracy(loader, model, device):
    correct, total = 0, 0
    # Track spam specifically
    spam_correct, spam_total = 0, 0
    model.eval()
    with torch.no_grad():
        for inputs, attn_mask, labels in loader:

            inputs, labels = inputs.to(device), labels.to(device)
            out = model(inputs)
            last_idx = attn_mask.sum(dim=1) - 1
            logits = out[torch.arange(out.size(0), device=device), last_idx, :]
            predicted = torch.argmax(logits, dim=-1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # Filter for Spam (Label 1)
            spam_mask = (labels == 1)
            spam_total += spam_mask.sum().item()
            spam_correct += (predicted[spam_mask] == labels[spam_mask]).sum().item()
    # Avoid division by zero
    spam_acc = spam_correct / spam_total if spam_total > 0 else 0.0
    global_acc = correct / total
    return global_acc, spam_acc


Using class weights: tensor([1.0000, 2.5428])


In [39]:
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    # Adding a progress bar for better visualization
    from tqdm import tqdm
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")
    
    for batch_idx, (inputs, attn_mask, targets) in progress_bar:

        inputs, targets = inputs.to(device), targets.to(device)

        # 9.1. Reset Gradients (of the `optimizer`)
        optimizer.zero_grad()

        # Forward Pass. The model outputs (batch, seq_len, vocab_size).
        # We only want the prediction for the LAST token in the sequence.
        out = model(inputs)  # (B, T, C)
        last_idx = attn_mask.sum(dim=1) - 1          # (B,)
        logits = out[torch.arange(out.size(0), device=device), last_idx, :]  # (B, C)

        # 9.2. Calculate the cross entropy loss
        loss = F.cross_entropy(logits, targets, weight=class_weights)

        # 9.3. Backward Pass
        loss.backward()
        
        # 9.4 Optimizer Step
        optimizer.step()

        # Update progress bar
        progress_bar.set_postfix(loss=loss.item())

    # 9.5 Add code to calculate the accuracy on train and test
    train_acc, train_spam_acc = calc_accuracy(train_loader, model, device)
    test_acc, test_spam_acc = calc_accuracy(test_loader, model, device)
    print(f"\nEpoch {epoch+1} Summary: Train Acc: {train_acc*100:.2f}% (Spam Acc: {train_spam_acc*100:.2f}%) | Test Acc: {test_acc*100:.2f}% (Spam Acc: {test_spam_acc*100:.2f}%)")


Epoch 1/3: 100%|██████████| 279/279 [18:10<00:00,  3.91s/it, loss=1.11] 



Epoch 1 Summary: Train Acc: 87.37% (Spam Acc: 19.10%) | Test Acc: 88.34% (Spam Acc: 25.33%)


Epoch 2/3: 100%|██████████| 279/279 [09:46<00:00,  2.10s/it, loss=0.221]



Epoch 2 Summary: Train Acc: 93.02% (Spam Acc: 56.11%) | Test Acc: 93.18% (Spam Acc: 59.33%)


Epoch 3/3: 100%|██████████| 279/279 [09:44<00:00,  2.09s/it, loss=0.215] 



Epoch 3 Summary: Train Acc: 95.98% (Spam Acc: 77.55%) | Test Acc: 95.70% (Spam Acc: 76.67%)


**Question 10**: 

Now run the cell above. You should see how the training loss changes after each batch (and epoch).
Describe thie trend: what do you see, is the model learning?

**Question 11 (optional)**: Change the number of epochs and/or the learning rate and/or the size of the training data, and investigate how the loss/accuracy of the model changes. You can do this editing and re-running the cells above, or creating new cells below.

**Question 12 (optional)**: Now test the model *on your own text*.

In [40]:
def classify_text(text, model, tokenizer, device, max_length=120, pad_token_id=50256):
    model.eval()
    
    # Encode the text
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    
    # Pad/Truncate 
    # (Matches the logic in SpamDataset so the model sees familiar input structures)
    encoded = encoded[:max_length]
    pad_len = max_length - len(encoded)
    encoded += [pad_token_id] * pad_len

    # Create tensor and add batch dimension
    encoded_tensor = torch.tensor(encoded).unsqueeze(0).to(device) # Shape: [1, max_length]

    # Get prediction
    with torch.no_grad():
        out = model(encoded_tensor)
        last_idx = (encoded_tensor != pad_token_id).sum(dim=1) - 1
        logits = out[torch.arange(out.size(0), device=device), last_idx, :]
    # Logits for the last token
        predicted_label = torch.argmax(logits, dim=-1).item()

    return "SPAM" if predicted_label == 1 else "NOT SPAM"

# --- TODO: Test the model ---
# Create 2 strings: one clearly spam, one normal.
text_1 = "Congratulations! You've been selected as a lucky winner of a brand new iPhone 18 Pro Max! To claim your prize, simply click on this link: [malicious-link.xyz/claim] and enter your bank details. Don't miss out, offer valid for 24 hours  only"
text_2 = "Hey, are we still on for dinner tonight at 7? Let me know!"

print(f"Text 1: {text_1} -> {classify_text(text_1, model, tokenizer, device)}")
print(f"Text 2: {text_2} -> {classify_text(text_2, model, tokenizer, device)}")

Text 1: Congratulations! You've been selected as a lucky winner of a brand new iPhone 18 Pro Max! To claim your prize, simply click on this link: [malicious-link.xyz/claim] and enter your bank details. Don't miss out, offer valid for 24 hours  only -> SPAM
Text 2: Hey, are we still on for dinner tonight at 7? Let me know! -> NOT SPAM
